# Extract, transform, load (ETL) procedure

In this exercise I will employ the ETL process using steps below

- extract world nations GDP data from [wikipedia](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)) (using BeautifulSoup),

- transform exctracted data (gdp figures from comma separated strings to numerical and then convert to desired currency using exchange rates obtained via the [currencylayer API](http://api.currencylayer.com/), and

- load the transformed data into a csv (or json) file.

In [1]:
# setup environment
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

## Extract GDP's

Extract countries/territories and their GDPs (US$ Million) from wikipedia

In [2]:
# function to get webpage text
def get_html_text(url: str) -> str: 
    '''fetches and returns webpage html text using requests'''
    html_text = requests.get(url).text
    return html_text

In [3]:
# function to create acronym of agency
def get_acronym(agency: str) -> str:
    '''given an agency, returns its acronym'''
    acronym = ''.join(word[0] for word in agency.split())
    return acronym

In [4]:
# function to get gdp data (from one agency)
def get_gdps(htmltext: str, acronym='WB') -> dict:
    '''given gdptablestext and agency acronym, fetches all
    country/territory names and corresponding gdp (in millionUS$)
    as per the agency and returns them as dictionary'''
    idx = acronyms.index(acronym) # get index
    gdps = {}
    for row in gdptablestext.find_all('tr')[1].find_all('tbody')[idx].find_all('tr'):
        cols = row.find_all('td')
        if len(cols) != 3:
            continue # skip non-relevant rows
        name = cols[1].find('a').text.strip()
        gdp = cols[2].text.strip() # US$million
        gdps[name] = gdp
            
    return gdps

In [5]:
# function to get gdp data (from all agencies)
def get_all_gdps(agencies: list, write=True) -> dict:
    '''utilizes function get_gdp to extract gdp data
    (from all agencies) and returns it as
    dictionary of dictionaries (also writes .json)'''
    all_gdps = {}
    for agency in agencies:
        acronym = get_acronym(agency)
        all_gdps[acronym] = get_gdps(gdptablestext,
                                         acronym=acronym)
    if write: # save to gdps.json file
        with open('gdps_usd.json', 'w') as wf:
            json.dump(all_gdps, wf, indent=4)
    
    return all_gdps

## Transform GDP's

In [6]:
# function to get currency rate data
def get_exchange_rates(url: str, write=True) -> dict:
    '''uses requests to fetch and return currency
    exchange rates (also writes .json file)'''
    import re
    # rates are in the `quotes` key of API response dictionary
    rates = requests.get(url).json()['quotes']
    # slice out leading 'USD' characters from rates keys
    #rates = {k[-3:]: v for k, v in rates.items()}
    # regular expression replace leading 'USD' group
    rates = {re.sub(r'^(USD)', r'', k): v for k, v in rates.items()}

    if write: # save exchange rates currency_exchange_rates.json
        with open('exchange_rates.json', 'w') as wf:
            json.dump(rates, wf, indent=4)
    
    return rates

In [7]:
# function to transform gdp data
def transform_gdps(gdps: dict, rates: dict,
                   currency='GBP') -> pd.DataFrame:
    '''transforms extracted gdp data from text to numerical,
       and converts (and returns) gdps
       from US$ to another currency of choice 
    '''
    #df = pd.read_json('gdps_usd.json') # if reading from json
    df = pd.DataFrame(gdps)
    # replace ',' from string numerals
    df.replace(',', '', regex=True, inplace=True)
    # convert columns to numeric, mark errors with NaN
    df = df.apply(pd.to_numeric, errors='coerce')
    # drop rows with NaN
    df.dropna(inplace=True)
    
    # convert gdp from (US$ Million) to specified currency
    rate = rates[currency]
    df = df.iloc[:] * rate
    df = df.round(0).astype(int) # vs. smallest signed int
    
    return df

## Load

In [8]:
# function to load transformed data to csv
def load(df_transformed: pd.DataFrame, wf_type='json'):
    '''loads transformed gdps into csv (or json) file'''
    joiner = currency.lower()
    if wf_type.lower() == 'json':
        df_transformed.to_json(f'gdps_{joiner}.json',
                               orient='columns', indent=4)
    elif wf_type.lower() == 'csv':
        df_transformed.to_csv(f'gdps_{joiner}.csv')

### Logging function

In [9]:
# function to log process
def log(message: str):
    '''function to help log procedure'''
    # Year-Monthname-Day-Hour-Minute-Second format
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open('gdps_logfile.txt', 'a') as af:
        af.write(f'{timestamp}, {message}\n')

## Execute the ETL procedue
and log it using the logging function

In [10]:
log('ETL Job Started')
log('Extract phase Started')

In [11]:
# wikipedia url for countries GDP
URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

html_text = get_html_text(URL)

# use BeautifulSoup to parse webpage contents
soup = BeautifulSoup(html_text, 'html.parser')

# gdp tables within table with class attribute 'wikitable' 
gdptablestext = soup.find('table', attrs={'class': 'wikitable'})

# list of agencies, each provides a gdp table
# from gdptables --> first tr --> all (3) td
agencies = [col.find('a').text.strip()
            for col in gdptablestext.find('tr').find_all('td')]

# list of acronyms of agencies that publish gdp data
acronyms = [get_acronym(agency) for agency in agencies]

# get gdps listed by all agencies
gdps = get_all_gdps(agencies)

In [12]:
# display top 10 nations GDP
df = pd.DataFrame(gdps)
today = datetime.now().date()
df.head(11).style.set_caption("GDP's (USD millions);" +
                    f' source: wikipedia {str(today)}')

,IMF,WB,UN
World,"93,863,851","87,798,526","87,445,260"
United States,"22,675,271","21,433,226","21,433,226"
China,"16,642,318","14,342,903","14,342,933"
Japan,"5,378,136","5,081,770","5,082,465"
Germany,"4,319,286","3,861,124","3,861,123"
United Kingdom,"3,124,650","2,829,108","2,826,441"
India,"3,049,704","2,868,929","2,891,582"
France,"2,938,271","2,715,518","2,715,518"
Italy,"2,106,287","2,003,576","2,003,576"
Canada,"1,883,487","1,736,426","1,741,496"


In [13]:
log('Extract phase Ended')
log('Transform phase Started')

In [14]:
# API url to call (for currency exchange rates)

# API base url
BASE_URL = 'http://api.currencylayer.com/'

# get currencylayer access key
with open("currencylayer_apikey.txt") as rf:
    API_KEY = str(rf.read().strip())

# complete url for API
URL = f'{BASE_URL}live?access_key={API_KEY}'

# get rates
#rates = get_exchange_rates(URL, write=True)
#'''
with open('exchange_rates.json') as rf:
    rates = json.load(rf)
#'''

# select currency to transform to 
currency = 'GBP'

# transform gdps
gdps_transformed = transform_gdps(gdps, rates,
                                  currency=currency)

In [15]:
# display top 10 nations GDP (transformed)
gdps_transformed.head(11).style.set_caption(
    f"GDP's ({currency} millions)")

,IMF,WB,UN
World,68277035,63865087,63608119
United States,16494106,15590636,15590636
China,12105705,10433099,10433121
Japan,3912083,3696505,3697010
Germany,3141870,2808601,2808600
United Kingdom,2272886,2057907,2055967
India,2218370,2086873,2103351
France,2137313,1975281,1975281
Italy,1532124,1457411,1457411
Canada,1370058,1263085,1266773


In [16]:
log('Transform phase Ended')
log("Load phase Started")

In [17]:
# Load transformed gdps to file

#load(gdps_transformed, wf_type='json')
load(gdps_transformed, wf_type='csv')

In [18]:
log("Load phase Ended")
log('ETL Job Ended')

Big thank you to IBM Developer Skills Network and [Python for Data Engineering Project](https://www.edx.org/course/python-for-data-engineering-project) course on EdX that taught me this methodology.